# --- Day 17: Clumsy Crucible ---

https://adventofcode.com/2023/day/17

## Parse the Input Data

In [1]:
def parse(filename):
    """Parse input data for puzzle.

    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.

    Returns
    -------
    city_map : list
        Each element (row) is a list of integers.
    """
    city_map = []
    with open(f'../inputs/{filename}.txt') as f:
        for line in f:
            city_map.append([int(c) for c in line.strip()])
    return city_map

In [2]:
parse("test_city_map")

[[2, 4, 1, 3, 4, 3, 2, 3, 1, 1, 3, 2, 3],
 [3, 2, 1, 5, 4, 5, 3, 5, 3, 5, 6, 2, 3],
 [3, 2, 5, 5, 2, 4, 5, 6, 5, 4, 2, 5, 4],
 [3, 4, 4, 6, 5, 8, 5, 8, 4, 5, 4, 5, 2],
 [4, 5, 4, 6, 6, 5, 7, 8, 6, 7, 5, 3, 6],
 [1, 4, 3, 8, 5, 9, 8, 7, 9, 8, 4, 5, 4],
 [4, 4, 5, 7, 8, 7, 6, 9, 8, 7, 7, 6, 6],
 [3, 6, 3, 7, 8, 7, 7, 9, 7, 9, 6, 5, 3],
 [4, 6, 5, 4, 9, 6, 7, 9, 8, 6, 8, 8, 7],
 [4, 5, 6, 4, 6, 7, 9, 9, 8, 6, 4, 5, 3],
 [1, 2, 2, 4, 6, 8, 6, 8, 6, 5, 5, 6, 3],
 [2, 5, 4, 6, 5, 4, 8, 8, 8, 7, 7, 3, 5],
 [4, 3, 2, 2, 6, 7, 4, 6, 5, 5, 5, 3, 3]]

## Part 1
---

In [3]:
adjacents = {
    "U" : {"deltas" : [(0, -1), (-1, 0), (0, 1)], "dirs" : ["L", "U", "R"]},
    "D" : {"deltas" : [(0, 1), (1, 0), (0, -1)],  "dirs" : ["R", "D", "L"]},
    "L" : {"deltas" : [(1, 0), (0, -1), (-1, 0)],  "dirs" : ["D", "L", "U"]},
    "R" : {"deltas" : [(-1, 0), (0, 1), (1, 0)], "dirs" :  ["U", "R", "D"]}
}

In [4]:
def possible(city_map, r, c, d, n):
    w = len(city_map[0]) - 1  # width/max col
    h = len(city_map) - 1  # height/max row

    # Gotta stay on the map
    if not 0 <= r <= h:
        return False
    elif not 0 <= c <= w:
        return False

    elif (
        (r == 0 and d == "D") or  # At row 0, there can't be any nodes heading in the down direction
        (r == h and d == "U") or  # At row height, there can't be any nodes heading in the up direction
        (c == 0 and d == "R") or  # At col 0, there can't be any nodes heading in the up right direction
        (c == w and d == "L")     # At col width, there can't be any nodes heading in the left direction
    ):
        return False

    # There are limits to how long you can be traveling in a direction, when you're near an edge...
    elif d == "R" and n > c + 1:
        return False
    elif d == "D" and n > r + 1:
        return False
    elif d == "L" and n > w - c + 1:
        return False
    elif d == "U" and n > h - r + 1:
        return False
    else:
        return True

In [5]:
def build_graph(city_map):
    """Build graph where each node is represented as:
        (pos, dir, num_steps_in_dir)
    """
    graph = {}

    for r, line in enumerate(city_map):
        for c, _ in enumerate(line):
            p = (r, c)
            for d in ["U", "R", "D", "L"]:
                for n in range(1, 4):
                    if possible(city_map, r, c, d, n):
                        graph[(p, d, n)] = {}

                        for _n in range(3):
                            delta = adjacents[d]["deltas"][_n]
                            _dir = adjacents[d]["dirs"][_n]

                            neigh = (p[0] + delta[0], p[1] + delta[1])
                            if possible(city_map, *neigh, _dir, _n):
                                if _dir == d:
                                    if n == 3:
                                        pass
                                    else:
                                        graph[(p, d, n)][(neigh, _dir, n+1)] = city_map[neigh[0]][neigh[1]]
                                else:
                                    graph[(p, d, n)][(neigh, _dir, 1)] = city_map[neigh[0]][neigh[1]]

    return graph

In [6]:
# Debuggin'
# build_graph(parse("test_city_map"))

In [7]:
import heapq

In [8]:
def solve1(filename, debug=False):
    city_map = parse(filename)
    graph = build_graph(city_map)

    start = ((0, 0), "U", 1)

    # Initialize all distances to infinity, except for the start node,
    # which is initialized to 0
    distances = {node: float('infinity') for node in graph}
    distances[start] = 0

    # Queue with priority scores (distances)
    priority_queue = [(0, start)]

    # Loop until priority queue is empty
    while priority_queue:
        # heapq, below, implements a binary heap, which is a data structure that
        # keeps elements in (priority) order, so the .heappop() method is
        # always taking off the *lowest* priority element in the queue (instaed)
        # of always taking the element based on insert order.
        current_distance, current_node = heapq.heappop(priority_queue)

        # If current distance is greater than the stored distance, skip
        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight

            # If a shorter path is found
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    max_row = len(city_map) - 1
    max_col = len(city_map[0]) - 1
    if debug: print([(k, v) for k, v in distances.items() if k[0] == (max_row, max_col)])

    return min([v for k, v in distances.items() if k[0] == (max_row, max_col)])

### Run on Test Data

In [9]:
solve1("test_city_map") == 102

True

### Run on Input Data

In [10]:
solve1("city_map")

785

## Part 2
---

### Run on Test Data

### Run on Input Data